## Zaman Serisi Tahmini 
## ALPEREN ERDEM 192511044

## Veri bilgisi
Burada kullanılan veriler Favorita mağazasının satışlarıdır.

## Özet

1. Verileri/kaynakları ve anlamlarını okuyun
2. Ortalama Satışlar, Mevsimsel Özellikler ve Petrol Fiyatları
3. Sezona, Mağazaya ve Ürün Türüne Göre Satışlar
4. Özellik oluşturma 
5. Model ve Tahminler
6. Gönderim/Gönderim

---

## Verileri/kaynakları ve anlamlarını okuyun

### Veri okuma

In [ ]:
path = '/kaggle/input/store-sales-time-series-forecasting/'
os.listdir(path)

data_oil=pd.read_csv(path+'oil.csv')
data_holidays_events=pd.read_csv(path+'holidays_events.csv')
data_stores=pd.read_csv(path+'stores.csv')
data_train=pd.read_csv(path+'train.csv',parse_dates=['date'])
data_test=pd.read_csv(path+'test.csv',parse_dates=['date'])
data_transactions=pd.read_csv(path+'transactions.csv')
df_oil = pd.read_csv(path+'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
df_stores = pd.read_csv(path+'stores.csv', header = 0)
df_trans = pd.read_csv(path+'transactions.csv', header = 0)
df_holidays = pd.read_csv(path+'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True)
df_sub = pd.read_csv(path+'sample_submission.csv', index_col='id')

### Özellikler ve anlamları

#### data_train

Ana veri kümesi. Satışları ve ilgili tarihleri ​​içerir.

1. id:  öğe kimliği
2. date: Kayıttaki tarih
3. store_nbr: mağaza numarası
4. family:ürün ailesi
5. sales: satış sayısı
6. onpromotion: belirli bir tarihte bir mağazada promosyonda olan bir ürün ailesindeki toplam öğe sayısı

In [ ]:
print(f'Eğitimde verilen örnek sayısı: {data_train.shape} \n')
print(data_train.info(),'\n')
print(data_train.head(),'\n')


print(f'Test için verilen numune sayısı: {data_test.shape} \n')
print(data_test.info(),'\n')
print(data_test.head(),'\n')

## Boş Verileri doldurma

 Eksik olan tarihleri belirlemek için kullanılır.

In [ ]:
## VERİ KAYIP ##
# '2013-01-01' ve '2017-08-31' arasındaki veri dizinini içeren bir DataFrame oluşturun
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

data_train_start = data_train['date'].min()
data_train_end = data_train['date'].max()

# Başlangıç_tarihi ile bitiş_tarihi arasında bir tarih aralığı oluşturun
date_range = pd.date_range(start=data_train_start,end=data_train_end)

# Veri aralıkları arasında farklı kullanımlar yapın ve geçerli verileri görüntülemek için satış verileri dışında kalan veriler olarak kullanınmissing_dates = date_range.difference(data_train['date'])

# Yanlış veriler
print("Güncel Veriler:")
# print(missing_dates)

In [ ]:
#Bu kod parçası, eğitim veri setindeki eksik verileri belirli yöntemlerle doldurur 
# ve ardından "id" sütununa doğrusal enterpolasyon uygular. İşte adım adım açıklaması:;

In [ ]:
data_train[['sales','onpromotion']] = data_train[['sales','onpromotion']].fillna(0.)

# "id" sütununa doğrusal enterpolasyon uygulayın
data_train.id = data_train.id.interpolate(method="linear")

#### Petrol Verileri

Petrol zar seti. Satışları etkileyen petrol fiyatlarındaki değişimi doğrulamak için kullanılır.

1. veri: veri
2. Petrol fiyatı: Belirli bir tarihe ait petrol fiyatı

In [ ]:
print(f'Yağlı Zar Numune Sayısı: {data_oil.shape} \n')
print(data_oil.head(),'\n')
print('BOŞ değerler \n',data_oil.isnull().sum(),'\n')


Veri kümesinin tek eksik kısmı *dcoilwtico* sütunundadır.

#### data_stores
Armazenar bilgileri

1. store_nbr: Mağaza numarası
2. city: Cidade da loja 
3. state: Mağazanın durumu
4. type: Loja tipi
5. cluster: Benzer mağazaların kümesi

In [ ]:
print(f'Örnek sayısı: {data_stores.shape} \n')
print(data_stores.head(),'\n')
print('BOŞ Değerler \n',data_stores.isnull().sum(),'\n')

#### data_holidays_events
İndirimlerin mevsimsel özelliklerine ilişkin bir açıklama oluşturmaya hizmet eden tatil bilgileri

1. date: Tatilin tarihi
2. type: Tatil türü
3. locale: Yerel veya bölgesel tatil
4. local_name: Şehir adı
5. description:Tatilin açıklaması
6. transfered: Tatilin başka bir tarihe devredilip devredilmediği

In [ ]:
print('BOŞ Değerler \n',data_holidays_events.isnull().sum(),'\n')

#### data_test

In [ ]:
print(f'Test zarı sayısı: {data_test.shape} \n')
data_test.head()
print('BOŞ Değerler \n',data_test.isnull().sum(),'\n')

### data_oil'deki boş girişleri doldurun

Aşağıda petrol fiyatlarının (*dcoilwtico*) zamana (*tarih*) göre grafiği.

In [ ]:
ax = data_oil.plot(x='date',y='dcoilwtico',title='Petrol Fiyatı',figsize=(18,10),linewidth=3)

ax.title.set_size(40)
plt.legend(fontsize=30)
plt.xlabel('Data',fontsize=20)
plt.ylabel('Fiyat $',fontsize=35)

Verilerde, mevcut girişler arasında enterpolasyonla doldurulabilecek çeşitli boşluklar vardır.

In [ ]:
# İki eğitim zarının başlangıcından iki test zarının sonuna kadar bir veri aralığı oluşturma
date_range = pd.date_range(data_train['date'].min(),data_train['date'].max())

date_df = pd.DataFrame({'date':date_range})

data_oil = pd.concat([date_df,data_oil.drop('date',axis=1)],axis = 1)

data_oil = data_oil.sort_values('date',ignore_index=True)

# Doğrusal enterpolasyon kullanarak eksik değerleri kontrol etme
data_oil.dcoilwtico = data_oil.dcoilwtico.interpolate(method="linear",limit_direction="both")

In [ ]:
ax = data_oil.plot(x='date',y='dcoilwtico',title='Petrol Fiyatı',figsize=(18,10),linewidth=3)

ax.title.set_size(40)
plt.legend(fontsize=30)
plt.xlabel('Data',fontsize=35)
plt.ylabel('Petrol $',fontsize=35)

Artık petrol fiyatı grafiğinde boşluk yok.

## Ortalama Satışlar, Mevsimsellik ve Petrol Fiyatları

### Ortalama haftalık/aylık satışlar
Satış grafiği ve haftalık, aylık ve yıllık ortalamaları.

In [ ]:
#data_train veri setindeki satış verilerini kullanarak çeşitli zaman aralıklarına göre ortalama satışları hesaplar 
# ve bu ortalamaları görselleştirir. Grafikte günlük, haftalık, aylık ve yıllık ortalama satışlar gösterilir. 

In [ ]:
mpl.rc('xtick',labelsize=30) 
mpl.rc('ytick',labelsize=30) 

average_sales = data_train.groupby('date').agg({'sales':'mean'}).reset_index()
avg_sales = average_sales.copy(deep=True)

avg_sales['Ortalama haftalık satışlar'] = avg_sales['sales'].ewm(span=7,adjust=False).mean()
avg_sales['Ortalama aylık satışlar'] = avg_sales['sales'].ewm(span=30,adjust=False).mean()
avg_sales['Ortalama yıllık satışlar'] = avg_sales['sales'].ewm(span=365,adjust=False).mean()

ax = avg_sales.plot(x='date',y=['sales','Ortalama haftalık satışlar','Ortalama aylık satışlar','Ortalama yıllık satışlar'],
                    title='Satışlar ve ortalama haftalık/aylık satışlar',figsize=(18,10),linewidth=3)
ax.title.set_size(40)
plt.legend(fontsize=30)
plt.xlabel('Data',fontsize=35)
plt.ylabel('Satış',fontsize=35)

Haftalık ve aylık ortalama eğrilerin birbirine çok benzerken yıllık ortalamanın oldukça farklı olması dikkat çekicidir. Son eğri ilk minimumdan güçlü bir şekilde etkilenir.


### Hareketli Ortalama ve Doğrusal Trend


#### Hareketli ortalama

Hareketli ortalama grafiği, serinin ne tür bir trende sahip olduğunu gösterir. Bu seri günlük gözlemlere sahip olduğundan, yıl içindeki kısa vadeli değişiklikleri düzeltmek için 365 günlük bir pencere seçmek iyi olur.

In [ ]:
moving_avg = average_sales.copy(deep = True)
moving_avg = moving_avg.set_index('date').to_period("D")

moving_average = moving_avg.rolling(
    window=365,       
    center=True,     
    min_periods=183,  
).mean()             

ax = moving_avg.plot(style=".",color="red")
moving_average.plot(ax=ax,linewidth=3,title="Mağaza içi satışlar: 365 günlük hareketli ortalama",legend=False,figsize=(18,10))

ax.title.set_size(40)
plt.legend(fontsize=30)
plt.xlabel('Data',fontsize=35)
plt.ylabel('Satış',fontsize=35)

#### Doğrusal Trend + 6 Aylık Tahmin

Satış verilerinizi doğrusal regresyon kullanarak uygun hale getirerek, satış eğiliminizin çok kaba bir tanımını elde edebilir ve ayrıca gelecekteki satışlara ilişkin bir ön tahmin alabilirsiniz.

In [ ]:
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

avg_sales_D = average_sales.copy(deep=True)
avg_sales_D = avg_sales_D.set_index('date').to_period("D")

dp = DeterministicProcess(
    index=avg_sales_D.index,  
    constant=True,      
    order=1,            
    drop=True,         
)
X = dp.in_sample()

y = avg_sales_D["sales"]  
model = LinearRegression(fit_intercept=False)
model.fit(X,y)

y_pred = pd.Series(model.predict(X),index=X.index)

X = dp.out_of_sample(steps=180)
y_fore = pd.Series(model.predict(X),index=X.index)

ax = avg_sales_D["2013-01":].plot(title="Satışlar: doğrusal eğilim + 6 aylık tahmin",**plot_params,figsize=(18,10))
ax = y_pred.plot(ax=ax,linewidth=5,label="Akım")
ax = y_fore.plot(ax=ax,linewidth=5,label="Trend tahmini",color="C3")
_ = ax.legend()

ax.title.set_size(40)
plt.legend(fontsize=20)
plt.xlabel('Data',fontsize=30)
plt.ylabel('Satış',fontsize=30)

 Bu kod parçası, sezonsal verilerin görselleştirilmesi için Matplotlib ve Seaborn kütüphanelerini kullanarak özel bir grafik stili ve fonksiyon tanımlar. 

In [ ]:
plt.style.use("seaborn-whitegrid")
plt.rc("figure",autolayout=True,figsize=(18,10))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

def seasonal_plot(X,y,period,freq,ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl",n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Sezonluk Porsiyon ({period}/{freq})")
    for line, name in zip(ax.lines,X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1,y_),
            xytext=(10,0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=20,
            va="center",
        )
    return ax

In [ ]:
seas_sales = average_sales.copy(deep=True)
seas_sales = seas_sales.set_index('date').to_period("D")

y = seas_sales["sales"]  
X = seas_sales.copy()

X['day'] = X.index.dayofweek 
X['week'] = X.index.week 

X['dayofyear'] = X.index.dayofyear
X['year'] = X.index.year


fig, (ax0,ax1) = plt.subplots(2,1,figsize=(11,6))

color_week = 'red'  
color_year = 'black'  

seasonal_plot(X,y="sales",period="week",freq="day",ax=ax0)
seasonal_plot(X,y="sales",period="year",freq="dayofyear",ax=ax1)

Yukarıdaki şekil (üst yarı), satışların haftanın ikinci yarısında arttığı net bir haftalık modeli gösteriyor.

#### Periodogram

Yukarıdaki mevsimsel grafikle aynı şekilde, periodogram zaman serisindeki periyodik özelliklerin olası varlığını vurgulamak için kullanılır.

In [ ]:
plot_periodogram(y);

Bazı belirgin mevsimsel özellikler vardır. Bunlardan en bariz olanı, yukarıdaki mevsimsel grafikte de öne sürülen haftalık mevsimselliktir. Ancak altı ayda bir ve iki haftada bir yaşanan zirveler gibi başka özellikler de var.

#### Sezonluk özellikler

Yukarıdaki özellikler Fourier terimi/terimleri ve regresyon algoritması için XGBoost ile modellenecektir. Amaç, işlevlerin satış verilerine mümkün olduğunca yakın bir tanımını elde etmektir.
Aşağıdaki görselde de görüldüğü gibi tahminler neredeyse orijinal verilerle örtüşüyor (ve aralarındaki fark çok küçük ve sıfır civarında dalgalanıyor).

In [ ]:
fourier = CalendarFourier(freq='M',order=4)

dp = DeterministicProcess(index=y.index,
        constant=True,
        order=1,
        seasonal=True,
        additional_terms=[fourier],
    drop=True,)

X =  dp.in_sample()

model = XGBRegressor().fit(X, y)
y_pred = pd.Series(
    model.predict(X),
    index=X.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X),index=X.index)
ax = y.plot(title="Ortalama Satışlar ve Mevsimsellikten Arındırılmış Satışlar",ylabel="items sold",color='black',linewidth=3)
ax = y_pred.plot(label="Mevsimsel",color='blue',linewidth=3)
ax = (y - y_pred).plot(label="Ortalama - Sezonluk",color='green',linewidth=3)
ax.legend();

ax.title.set_size(40)
plt.legend(fontsize=30)
plt.xlabel('Data',fontsize=30)
plt.ylabel('Satış',fontsize=30)

In [ ]:
y_deseason = y - y_pred

fig, (ax1, ax2) = plt.subplots(2,1,sharex=True,sharey=True,figsize=(13,10))
ax1 = plot_periodogram(y,ax=ax1)
ax1.set_title("Ürün Satış Sıklığı Bileşenleri")
ax2 = plot_periodogram(y_deseason,ax=ax2);
ax2.set_title("Sezonluk");

#### Ortalama Satış Tahmini

Aşağıda 180 günlük ortalama satış tahminini sunuyorum. Tahmin, doğrusal regresyonla elde edilen tahminle karşılaştırılabilir

In [ ]:
# 180-Day Forecast
X = dp.out_of_sample(steps=180)
y_fore = pd.Series(model.predict(X),index=X.index)

y_pred = pd.Series(model.predict(X),index=X.index)
ax = y.plot(title="Ortalama Satış Tahmini",ylabel="items sold",color='blue',linewidth=3)
ax = y_fore.plot(label="180 günlük tahmin",color='black',linewidth=3)
ax.legend();

ax.title.set_size(40)
plt.legend(fontsize=30)
plt.xlabel('Data',fontsize=30)
plt.ylabel('Vendas',fontsize=30)

#### Bayram

Aşağıda gösterildiği gibi ana tatil türleri “standart” tatillerdir. Bunların yarısı ulusal bayramlardır.

In [ ]:

blue_shades = ['#cce7ff', '#99ceff', '#66b5ff', '#339cff', '#0083ff']

def label_function(val):
    return f'{val:.0f}%'

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))

data_holidays_events.groupby('type').size().plot(
    kind='pie', 
    autopct=label_function, 
    textprops={'fontsize': 11}, 
    ax=ax1,
    colors=blue_shades[:data_holidays_events['type'].nunique()]
)

data_holidays_events.groupby('locale').size().plot(
    kind='pie', 
    autopct=label_function, 
    textprops={'fontsize': 20}, 
    ax=ax2,
    colors=blue_shades[:data_holidays_events['locale'].nunique()]
)

ax1.set_ylabel('Tipo de feriado', size=22)
ax2.set_ylabel('Nacional/Local', size=22)
plt.tight_layout()
plt.show()

Yalnızca 'Tatil' tatil türünü içeren başka bir sütun oluşturacağım. 'Köprü', 'Ek'... ile ilgili girişler dahil edilmeyecektir.Criarei outra coluna que contém apenas o tipo de feriados 'Férias'. As entradas correspondentes a 'Ponte', 'Adicional' ... não serão incluídas.

In [ ]:

data_holidays_events['real holidays'] = np.where(data_holidays_events['type'] == 'Holiday',True,False)

holiday_list = data_holidays_events[data_holidays_events['real holidays'] == True]['date'].tolist()

for day in holiday_list:
    day = datetime.datetime.strptime(day,'%Y-%m-%d')

data_holidays_events = data_holidays_events.drop(['real holidays'],axis=1)

data_holidays_events['holy holidays'] = pd.Series(holiday_list)

data_holidays_events.head()

### Petrol Fiyatı vs Satışlar vs Promosyonlar

#### Petrol fiyatı ve satışlar

Petrol fiyatındaki değişimin ortalama satışları etkileyip etkilemeyeceğini görmek ilginç olacak. Bunu yapmak için ortalama satışları petrol fiyatlarına göre dağılım grafiğine çiziyorum. Noktaların doğrusal uyumu, yaklaşık da olsa, petrol fiyatı arttıkça satış sayısının azaldığını gösteriyor (tuhaf).

#### Petrol fiyatı ve promosyonlar

Ayrıca petrol fiyatlarındaki iniş ve çıkışların satıştaki toplam ürün sayısını nasıl etkilediğini de görebilirsiniz.

In [ ]:
data_prom_oil = data_train.copy()

data_prom_oil = data_train.merge(data_oil,on=["date"],how="left")

data_prom_oil = data_prom_oil.groupby(['dcoilwtico'],as_index=False)['onpromotion'].mean()
plt.figure(figsize=(14,8),dpi=80)
sns.regplot(x=data_prom_oil.dcoilwtico,y=data_prom_oil.onpromotion)
plt.ylabel("Itens em promoção",fontsize=25)
plt.title("Preço do petróleo vs promoções",fontsize=30)
plt.xticks(size=20)
plt.yticks(size=20)
plt.xlabel('Preço do petróleo',fontsize=25)
plt.show();

Yukarıdaki grafik ve onun doğrusal enterpolasyonu, mağazaların yalnızca petrol fiyatı orta/düşük olduğunda ürün tanıtımı yaptığını göstermektedir.

#### Satış x Promosyonlar

In [ ]:
data_sales_prom = data_train.copy()

data_sales_prom = data_train.merge(data_oil,on=["date"],how="left")

data_sales_prom = data_sales_prom.groupby(['onpromotion'],as_index=False)['sales'].mean()
plt.figure(figsize=(14,8),dpi=80)
sns.regplot(x=data_sales_prom.onpromotion,y=data_sales_prom.sales)
plt.ylabel("Vendas médias",fontsize=25)
plt.title("Promoções vs Vendas",fontsize=30)
plt.xticks(size=20)
plt.yticks(size=20)
plt.xlabel('Itens em promoção',fontsize=25)
plt.show();

#### Satış x Promosyonlar

## Sezona, Mağazaya ve Ürün Türüne Göre Satışlar

### Sezona göre satışlar

In [ ]:
data_train['date'].min(), data_train['date'].max()

In [ ]:
data_train['year'] = [d.year for d in data_train.date]
data_train['month'] = [d.strftime('%b') for d in data_train.date]
data_train['day'] = [d.dayofweek for d in data_train.date]

data_train["season"] = np.where(data_train.date.dt.month.isin([1,2,3]),'JAN-MAR',1)
data_train["season"] = np.where(data_train.date.dt.month.isin([4,5,6]),'ABR-JUN',data_train["season"])
data_train["season"] = np.where(data_train.date.dt.month.isin([7,8,9]),'JUL-SET',data_train["season"])
data_train["season"] = np.where(data_train.date.dt.month.isin([10,11,12]),'OUT-DEZ',data_train["season"])

In [ ]:
sns.set(rc = {'figure.figsize':(20,20),'axes.labelsize':35}) 
sns.set(font_scale=2)

fig,axes = plt.subplots(4,1,dpi=50)
sns.boxplot(x='year',y='sales',data=data_train,ax=axes[0])
sns.boxplot(x='season',y='sales',data=data_train,ax=axes[1])
sns.boxplot(x='month',y='sales',data=data_train,ax=axes[2])
sns.boxplot(x='day',y='sales',data=data_train,ax=axes[3])

axes[0].set_title('Ortalama satışlar (yıllık)',fontsize=25); 
axes[1].set_title('Ortalama satışlar (dönem başına)',fontsize=25);
axes[2].set_title('Ortalama satışlar (aylık)',fontsize=25);
axes[3].set_title('Ortalama satışlar (günlük)',fontsize=25)

plt.show()

### Mağazaya göre satışlar

Satışları bireysel mağazalara göre sınıflandırmak için *data_stores* ve *data_train* veri kümelerini birleştirmem gerekiyor.

In [ ]:
data_store_sales = data_train.copy()

data_store_sales = data_train.merge(data_stores,on=["store_nbr"],how="left")

data_store_sales.head()

#### Şehir, Eyalet ve Mağaza Numarasına Göre Satışlar

In [ ]:
sales_city = data_store_sales.groupby(['city'])['sales'].sum()
sales_city.plot.bar(x='Şehir',y='Satış',ylabel='Rekor (toplam satışlar)',logy=True,title='Şehir satışları',color=['#e0f7fa', '#80deea', '#26c6da', '#00acc1', '#007c91'])

In [ ]:
sales_state = data_store_sales.groupby(['state'])['sales'].sum()
sales_state.plot.bar(x='durum',y='Satış',ylabel='Kayıtlar (toplam satışlar)',logy=True,title='Eyalete göre satışlar',color=['#e8f5e9', '#a5d6a7', '#66bb6a', '#43a047', '#1b5e20'])

### Makale türlerine göre satışlar

Tüm ürün kategorileri.### Ürün türlerine göre satışlar

Tüm ürün kategorileri.

In [ ]:
print(data_store_sales['family'].unique())

Aşağıda satışları makale türüne göre çiziyorum. Verileri daha okunaklı hale getirmek için logaritmik bir ölçek kullanıyorum.Below I am plotting the sales by article type. I am using a logarithmic scale to make the data more readable.

In [ ]:
sales_type = data_store_sales.groupby(['family'])['sales'].sum()
sales_type.plot.bar(x='Tip',y='Satış',ylabel='Kayıtlar (toplam satışlar)',logy=True,title='Ürün türüne göre satışlar',color=['#440154', '#482173', '#433E85', '#38598C', '#2D708E', '#238A8D', '#20A387', '#3DBC74', '#73D055', '#B4DE2C'])

In [ ]:
sales_prom = data_store_sales.groupby(['family'])['onpromotion'].sum()
sales_prom.plot.bar(x='Tip',y='Terfi',logy=True,ylabel='Kayıtlar (Üründeki promosyonların sayısı)',title='Ürün türüne göre promosyonlar',color=['#0d0887', '#6a00a8', '#b12a90', '#e16462', '#fca636','#fee724', '#b4de2c', '#5ec962', '#21918c', '#440154'])

## Merge Datasets

In [ ]:
df_oil['ma_oil'] = df_oil['dcoilwtico'].rolling(7).mean()

calendar = calendar.merge(df_oil, how='left', left_index=True, right_index=True)

calendar['ma_oil'].fillna(method='bfill', inplace=True)

calendar['dofw'] = calendar.index.dayofweek

In [ ]:
df_holidays['date'] = df_holidays['date'].replace({'2013-04-29': pd.to_datetime('2013-03-29')})

df_holidays = df_holidays.set_index('date').sort_index()

df_holidays = df_holidays[df_holidays.locale == 'National']

df_holidays = df_holidays.groupby(df_holidays.index).first()



In [ ]:
calendar['wd'] = True

calendar.loc[calendar.dofw > 4, 'wd'] = False

calendar = calendar.merge(df_holidays, how='left', left_index=True, right_index=True)

calendar.loc[calendar.type == 'Bridge', 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False

calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False

calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True), 'wd'] = True

In [ ]:
calendar['dcoilwtico'].fillna(method='bfill', inplace=True)
calendar['ma_oil'].fillna(method='bfill', inplace=True)

calendar.head()

calendar.isnull().sum()

In [ ]:
df_train = pd.read_csv(path+'train.csv',usecols=['store_nbr', 'family', 'date', 'sales'],dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

df_test = pd.read_csv(path+'test.csv', usecols=['store_nbr', 'family', 'date'], dtype={'store_nbr': 'category', 'family': 'category'}, parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

sdate = '2017-04-01'
edate = '2017-08-15'

* ## Mağaza Verilerinin Sıralanması ve Mevsimsel Analiz

### Dönem Seçimi
Analizimiz için 1 Nisan 2017 ile 15 Ağustos 2017 arasındaki aralığa odaklanıyoruz. Bu dönem, satışlardaki belirli kalıpları incelemek için seçildi.

### Satış Verilerinin Hazırlanması
Satış verilerini (df_train) mağaza ve ürün kategorisine ("aile") göre düzenliyoruz. Bu, her mağaza ve kategorideki trendleri ve kalıpları tanımlamayı kolaylaştırır.

### Mevsimsel Kalıpların Analizi
Haftalık satışlardaki mevsimsel kalıpları tespit etmek için Fourier Dönüşümü'nü kullanıyoruz. Bu, satışların zaman içinde nasıl değiştiğini anlamamıza ve tekrarlanan döngüleri tanımlamamıza yardımcı olur.

### Deterministik Süreç
Satışların zamansal değişimini ve mevsimsel kalıplarını yakalamak için deterministik bir model geliştirdik. Bu model, satışlardaki değişiklikleri tahmin etmek için Fourier Dönüşümü tarafından elde edilen bilgileri kullanır.

### Dışsal Değişkenlerin Oluşturulması
Deterministik modelle, hava durumu ve mevsimsel kalıplar gibi satışları etkileyebilecek dış faktörleri temsil eden dışsal değişkenler yaratıyoruz. Bu değişkenler tahmin modelinin doğruluğunun arttırılmasına yardımcı olur.

### Özet
Özetle yaklaşımımız, bir dönemi dikkatli bir şekilde seçmeyi, satış verilerini sistematik olarak organize etmeyi, Fourier Dönüşümü ile mevsimsel kalıpları analiz etmeyi, deterministik bir model geliştirmeyi ve seçilen dönemde mağazalar ve kategorilerdeki satışları doğru bir şekilde tahmin etmek için dışsal değişkenler oluşturmayı içerir.

In [ ]:
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]

fourier = CalendarFourier(freq='W', order=4)

dp = DeterministicProcess(index=y.index, constant=False, order=1, seasonal=False, additional_terms=[fourier],drop=True)

X = dp.in_sample()

X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values

X['dofw'] = calendar.loc[sdate:edate]['dofw'].values

X['wd']   = calendar.loc[sdate:edate]['wd'].values

X['type'] = calendar.loc[sdate:edate]['type'].values


X = pd.get_dummies(X, columns=['dofw'], drop_first=True)


X = pd.get_dummies(X, columns=['type'], drop_first=False)

### SatışRegresörü

Tahmin için tahmin yöntemi, yeni öngörücü değişkenlere dayalı tahminler yapmak için düzeltilmiş modelleri kullanır. Tahminler paralel olarak yürütülerek süreçte verimlilik ve hız sağlanır. Nihai sonuç, her sütunun belirli bir mağaza ve ürün ailesi kombinasyonu için bir modelin tahminlerini temsil ettiği bir tahmin matrisidir. Kısacası SalesRegressor, farklı ürün kategorilerine uygun paralel işleme ve makine öğrenimi tekniklerini kullanarak performansı optimize ederek doğru ve kişiselleştirilmiş satış tahmini modelleri oluşturmayı kolaylaştırır.

In [ ]:
class SalesRegressor:
   

    def __init__(self, num_jobs=-1, verbosity=0):
     
        self.n_jobs = num_jobs
        self.verbose = verbosity
        self.estimators_ = None  

    def _train_model(self, X, y):
       

        warnings.simplefilter(action='ignore', category=FutureWarning)

        if y.name[2] == 'SCHOOL AND OFFICE SUPPLIES':
    
            model = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=1)
        else:
      
            model = Ridge(fit_intercept=True, solver='auto', alpha=0.5)

        model.fit(X, y)  
        return model 

    def fit(self, X, y):

        self.estimators_ = Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(
            delayed(self._train_model)(X, y.iloc[:, i]) for i in range(y.shape[1])
        )
        return self  

    def predict(self, X):

        y_pred = Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(
            delayed(model.predict)(X) for model in self.estimators_
        )
        return np.stack(y_pred, axis=1)  


    
model = SalesRegressor(num_jobs=-1, verbosity=0)
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [ ]:
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()

y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.)  

y_target.groupby('family').apply(lambda x: mean_squared_log_error(x['sales'], x['sales_pred']))

## Öngörülen Satışlar ve Gerçekleşen Satış Tablosu

In [ ]:
df_2017 = df_train[df_train.index.get_level_values('date').year == 2017]

y_pred_2017 = y_target[y_target['date'].dt.year == 2017]

df_2017_dates = df_2017.index.get_level_values('date').to_timestamp()

plt.figure(figsize=(12, 8))
plt.plot(df_2017_dates, df_2017['sales'], label='Gerçek Satışlar')
plt.plot(y_pred_2017['date'], y_pred_2017['sales_pred'], label='Öngörülen Satışlar')
plt.xlabel('Data')
plt.ylabel('Satış')
plt.title('Öngörülen Satışlar vs. Gerçek Satışlar - 2017 Yılı')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
stest = '2017-08-16'
etest = '2017-08-31'

X_test = dp.out_of_sample(steps=16)

X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test['dofw'] = calendar.loc[stest:etest]['dofw'].values
X_test['wd']   = calendar.loc[stest:etest]['wd'].values
X_test = pd.get_dummies(X_test, columns=['dofw'], drop_first=True)

X_test[['type_Additional', 'type_Event', 'type_Holiday', 'type_Transfer']] = 0
sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])
sales_pred[sales_pred < 0] = 0.


In [ ]:
df_sub.sales = sales_pred.values
df_sub.to_csv('./submission.csv')